In [1]:
import gym

In [2]:
env = gym.make("CartPole-v1")
obs = env.reset()
obs

array([ 0.01435374,  0.015396  , -0.03516366,  0.03883541])

In [3]:
env.render()

True

In [14]:
env.close()

In [5]:
env.action_space

Discrete(2)

In [6]:
action=1
obs, reward, done, info = env.step(action)
obs

array([ 0.01466166,  0.21100409, -0.03438695, -0.26473134])

In [7]:
reward

1.0

In [8]:
done

False

In [9]:
info

{}

In [10]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

In [12]:
totals = []
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(200):
        action = basic_policy(obs)
        obs ,reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

In [13]:
import numpy as np
np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

(42.62, 8.86947574549928, 24.0, 72.0)

In [18]:
import tensorflow as tf
from tensorflow import keras

n_inputs = 4

model = keras.models.Sequential([
    keras.layers.Dense(5, activation="elu", input_shape=[n_inputs]),
    keras.layers.Dense(1, activation="sigmoid")
])